# Market Basket Analysis with Apriori
***
Using [Online Retail Data Set](http://archive.ics.uci.edu/ml/datasets/Online+Retail)

**Data Set Information:**

This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

**Attribute Information:**

- InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
- UnitPrice: Unit price. Numeric, Product price per unit in sterling.
- CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal, the name of the country where each customer resides.

**Source:**

Daqing Chen, Sai Liang Sain, and Kun Guo, Data mining for the online retail industry: A case study of RFM model-based customer segmentation using data mining, Journal of Database Marketing and Customer Strategy Management, Vol. 19, No. 3, pp. 197â€“208, 2012 (Published online before print: 27 August 2012. doi: 10.1057/dbm.2012.17).


In [1]:
# Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
pd.options.display.max_columns = None
pd.options.display.max_rows = 15

We reduced the row from original data and take only about 50% of them, because we will only do this intuitively and so that our local machine can run the dataset.

In [2]:
# Load the data
data = pd.read_excel(r'./dataset/online_retail_reduced.xlsx')

In [3]:
print(data.shape)
display(data.head(10))

(104400, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [4]:
# Drop Invoice that start with letter c
data['InvoiceNo'] = data['InvoiceNo'].apply(str)
data = data[~data['InvoiceNo'].str.contains("C")]

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102497 entries, 0 to 104399
Data columns (total 8 columns):
InvoiceNo      102497 non-null object
StockCode      102497 non-null object
Description    102194 non-null object
Quantity       102497 non-null int64
InvoiceDate    102497 non-null datetime64[ns]
UnitPrice      102497 non-null float64
CustomerID     67008 non-null float64
Country        102497 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 7.0+ MB


In [6]:
# Clean the description column
data['Description'] = data['Description'].str.strip()

In [7]:
# Again for efficiency we only take small sample from the dataset, which is the transaction in Frace
basket = (data[data['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

# One hot encode the product columns, to acceptable format for apriori function
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [8]:
display(basket_sets)

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,20 DOLLY PEGS RETROSPOT,3 HOOK HANGER MAGIC GARDEN,3 PIECE SPACEBOY COOKIE CUTTER SET,3 TRADITIONAl BISCUIT CUTTERS SET,4 PINK DINNER CANDLE SILVER FLOCK,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER MAGIC TOADSTOOL,5 HOOK HANGER RED MAGIC TOADSTOOL,6 RIBBONS EMPIRE,6 RIBBONS RUSTIC CHARM,60 CAKE CASES DOLLY GIRL DESIGN,72 SWEETHEART FAIRY CAKE CASES,AGED GLASS SILVER T-LIGHT HOLDER,AIRLINE BAG VINTAGE JET SET BROWN,AIRLINE BAG VINTAGE JET SET RED,ALARM CLOCK BAKELIKE CHOCOLATE,ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE IVORY,ALARM CLOCK BAKELIKE ORANGE,ALARM CLOCK BAKELIKE PINK,ALARM CLOCK BAKELIKE RED,ANTIQUE ALL GLASS CANDLESTICK,ANTIQUE GLASS DRESSING TABLE POT,ANTIQUE GLASS PEDESTAL BOWL,ANTIQUE SILVER TEA GLASS ENGRAVED,APPLE BATH SPONGE,AREA PATROLLED METAL SIGN,ASS COLOUR GLOWING TIARAS,ASSORTED BOTTLE TOP MAGNETS,ASSORTED COLOUR BIRD ORNAMENT,ASSORTED COLOUR MINI CASES,ASSORTED COLOURS SILK FAN,ASSORTED EASTER DECORATIONS BELLS,ASSORTED EASTER GIFT TAGS,ASSTD DESIGN 3D PAPER STICKERS,BABUSHKA LIGHTS STRING OF 10,BAG 125g SWIRLY MARBLES,BAKING MOULD CHOCOLATE CUPCAKES,BAKING MOULD EASTER EGG MILK CHOC,BAKING MOULD EASTER EGG WHITE CHOC,BAKING SET 9 PIECE RETROSPOT,BAKING SET SPACEBOY DESIGN,BALLOON PUMP WITH 10 BALLOONS,BALLOON WATER BOMB PACK OF 35,BALLOONS WRITING SET,BASKET OF TOADSTOOLS,BATH BUILDING BLOCK WORD,BIG DOUGHNUT FRIDGE MAGNETS,BINGO SET,BIRD DECORATION RED RETROSPOT,BIRD HOUSE HOT WATER BOTTLE,BLACK CANDELABRA T-LIGHT HOLDER,BLACK LOVE BIRD CANDLE,BLACK RECORD COVER FRAME,BLACK/BLUE POLKADOT UMBRELLA,BLUE 3 PIECE POLKADOT CUTLERY SET,BLUE GEISHA GIRL,BLUE HARMONICA IN BOX,BLUE PARTY BAGS,BLUE POLKADOT CUP,BLUE ROSE FABRIC MIRROR,BLUE SCANDINAVIAN PAISLEY WRAP,BLUE TEA TOWEL CLASSIC DESIGN,BOOM BOX SPEAKER BOYS,BOOM BOX SPEAKER GIRLS,BOTTLE BAG RETROSPOT,BOX OF 24 COCKTAIL PARASOLS,BOX OF VINTAGE JIGSAW BLOCKS,BOYS VINTAGE TIN SEASIDE BUCKET,BREAD BIN DINER STYLE RED,BRIGHT BLUES RIBBONS,BUTTERFLIES STICKERS,CABIN BAG VINTAGE RETROSPOT,CAKE PLATE LOVEBIRD PINK,CAKE PLATE LOVEBIRD WHITE,CAKE STAND 3 TIER MAGIC GARDEN,CAKE STAND WHITE TWO TIER LACE,CALENDAR PAPER CUT DESIGN,CANDY SHOP STICKER SHEET,CARD BIRTHDAY COWBOY,CARD CIRCUS PARADE,CARD HOLDER GINGHAM HEART,CARD I LOVE LONDON,CAROUSEL DESIGN WASHBAG,CARRIAGE,CARTOON PENCIL SHARPENERS,CERAMIC CHERRY CAKE MONEY BANK,CERAMIC HEART FAIRY CAKE MONEY BANK,CERAMIC PIRATE CHEST MONEY BANK,CERAMIC STRAWBERRY CAKE MONEY BANK,CERAMIC STRAWBERRY MONEY BOX,CERAMIC STRAWBERRY TRINKET TRAY,CHARLOTTE BAG DOLLY GIRL DESIGN,CHARLOTTE BAG PINK POLKADOT,CHARLOTTE BAG SUKI DESIGN,CHERRY BLOSSOM DECORATIVE FLASK,CHILDREN'S APRON DOLLY GIRL,CHILDREN'S CIRCUS PARADE MUG,CHILDREN'S SPACEBOY MUG,CHILDRENS APRON APPLES DESIGN,CHILDRENS APRON SPACEBOY DESIGN,CHILDRENS DOLLY GIRL MUG,CHILDRENS GARDEN GLOVES BLUE,CHILDRENS GARDEN GLOVES PINK,CHILDS BREAKFAST SET CIRCUS PARADE,CHILDS BREAKFAST SET DOLLY GIRL,CHILDS BREAKFAST SET SPACEBOY,CHILDS GARDEN BRUSH BLUE,CHILDS GARDEN BRUSH PINK,CHILDS GARDEN FORK BLUE,CHILDS GARDEN FORK PINK,CHILDS GARDEN RAKE BLUE,CHILDS GARDEN RAKE PINK,CHILDS GARDEN SPADE BLUE,CHILDS GARDEN SPADE PINK,CHILDS GARDEN TROWEL BLUE,CHILDS GARDEN TROWEL PINK,CHINESE DRAGON PAPER LANTERNS,CHOCOLATE BOX RIBBONS,CHOCOLATE THIS WAY METAL SIGN,CHRISTMAS METAL TAGS ASSORTED,CHRISTMAS RETROSPOT ANGEL WOOD,CIRCUS PARADE CHILDRENS EGG CUP,CIRCUS PARADE LUNCH BOX,CLASSICAL ROSE SMALL VASE,CLOTHES PEGS RETROSPOT PACK 24,COFFEE MUG APPLES DESIGN,COFFEE MUG CAT + BIRD DESIGN,COFFEE MUG DOG + BALL DESIGN,COFFEE MUG PEARS DESIGN,COWBOYS AND INDIANS BIRTHDAY CARD,CREAM FELT EASTER EGG BASKET,CREAM SWEETHEART EGG HOLDER,CROCHET ROSE PURSE WITH SUEDE BACK,CUPCAKE LACE PAPER SET 6,CURIOUS IMAGES GIFT WRAP SET,CUTE CATS TAPE,DAIRY MAI

In [9]:
# Import the function
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

In [10]:
# Show the itemsets that have atleast 0.2 support values
display(frequent_itemsets)

,support,itemsets
0,0.072464,(10 COLOUR SPACEBOY PEN)
1,0.101449,(4 TRADITIONAL SPINNING TOPS)
2,0.086957,(ALARM CLOCK BAKELIKE GREEN)
3,0.072464,(ALARM CLOCK BAKELIKE ORANGE)
4,0.086957,(ALARM CLOCK BAKELIKE PINK)
5,0.086957,(ALARM CLOCK BAKELIKE RED)
6,0.086957,(ASSORTED COLOUR MINI CASES)
...,...,...
174,0.115942,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED..."
175,0.072464,"(ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI..."


In [11]:
# Create the association rules using lift metric
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [12]:
# Show the rules that have atleast 1 lift value
display(rules)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE ORANGE),(ALARM CLOCK BAKELIKE GREEN),0.072464,0.086957,0.072464,1.000000,11.500000,0.066163,inf
1,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE ORANGE),0.086957,0.072464,0.072464,0.833333,11.500000,0.066163,5.565217
2,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.086957,0.086957,0.086957,1.000000,11.500000,0.079395,inf
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.086957,0.086957,0.086957,1.000000,11.500000,0.079395,inf
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.086957,0.086957,0.086957,1.000000,11.500000,0.079395,inf
5,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.086957,0.086957,0.086957,1.000000,11.500000,0.079395,inf
6,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE ORANGE),0.086957,0.072464,0.072464,0.833333,11.500000,0.066163,5.565217
...,...,...,...,...,...,...,...,...,...
365,"(RED RETROSPOT MINI CASES, STRAWBERRY LUNCH BO...","(PLASTERS IN TIN WOODLAND ANIMALS, PLASTERS IN...",0.086957,0.144928,0.072464,0.833333,5.750000,0.059861,5.130435
366,"(RED RETROSPOT MINI CASES, PLASTERS IN TIN SPA...","(PLASTERS IN TIN WOODLAND ANIMALS, STRAWBERRY ...",0.101449,0.130435,0.072464,0.714286,5.476190,0.059231,3.043478


From above result we can know the set of items that will likely be bought together, let's filter them more by the confidence

In [20]:
rules.sort_values(by=['confidence', 'lift'], ascending=False).head(15)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE ORANGE),(ALARM CLOCK BAKELIKE GREEN),0.072464,0.086957,0.072464,1.0,11.5,0.066163,inf
2,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.086957,0.086957,0.086957,1.0,11.5,0.079395,inf
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.086957,0.086957,0.086957,1.0,11.5,0.079395,inf
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.086957,0.086957,0.086957,1.0,11.5,0.079395,inf
5,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.086957,0.086957,0.086957,1.0,11.5,0.079395,inf
7,(ALARM CLOCK BAKELIKE ORANGE),(ALARM CLOCK BAKELIKE PINK),0.072464,0.086957,0.072464,1.0,11.5,0.066163,inf
8,(ALARM CLOCK BAKELIKE ORANGE),(ALARM CLOCK BAKELIKE RED),0.072464,0.086957,0.072464,1.0,11.5,0.066163,inf
10,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.086957,0.086957,0.086957,1.0,11.5,0.079395,inf
11,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.086957,0.086957,0.086957,1.0,11.5,0.079395,inf
138,"(ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...",(ALARM CLOCK BAKELIKE GREEN),0.072464,0.086957,0.072464,1.0,11.5,0.066163,inf


From above result we can conclude that the when a customer buy an alarm clock, they will buy another one with a different color.
***
Now we should see customers behavior in another country. Let's try Japan

In [54]:
data['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Finland',
       'Austria', 'Bahrain', 'Israel', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic'], dtype=object)

In [29]:
basket2 = (data[data['Country'] =="Belgium"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.07, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

In [30]:
rules2.sort_values(by=['confidence', 'lift'], ascending=False).head(15)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(BLUE POLKADOT CUP),(BLUE POLKADOT BOWL),0.125,0.125,0.125,1.0,8.0,0.109375,inf
1,(BLUE POLKADOT BOWL),(BLUE POLKADOT CUP),0.125,0.125,0.125,1.0,8.0,0.109375,inf
4,(BLUE POLKADOT BOWL),(PINK POLKADOT CUP),0.125,0.125,0.125,1.0,8.0,0.109375,inf
5,(PINK POLKADOT CUP),(BLUE POLKADOT BOWL),0.125,0.125,0.125,1.0,8.0,0.109375,inf
6,(BLUE POLKADOT BOWL),(PINK POLKADOT BOWL),0.125,0.125,0.125,1.0,8.0,0.109375,inf
7,(PINK POLKADOT BOWL),(BLUE POLKADOT BOWL),0.125,0.125,0.125,1.0,8.0,0.109375,inf
12,(BLUE POLKADOT CUP),(PINK POLKADOT CUP),0.125,0.125,0.125,1.0,8.0,0.109375,inf
13,(PINK POLKADOT CUP),(BLUE POLKADOT CUP),0.125,0.125,0.125,1.0,8.0,0.109375,inf
14,(BLUE POLKADOT CUP),(PINK POLKADOT BOWL),0.125,0.125,0.125,1.0,8.0,0.109375,inf
15,(PINK POLKADOT BOWL),(BLUE POLKADOT CUP),0.125,0.125,0.125,1.0,8.0,0.109375,inf


From above results we can see the different trend between French and Belgian. Belgian tend to buy Cup and Bowl at the same time and mix the colors.